In [58]:
import numpy as np
import seaborn as sns
import struct

In [66]:
import  pymysql
import  pymysql.cursors

db = pymysql.connect(host='localhost',
                           user='root',
                           password='xzx970413',
                           db='Wechat_inf',
                           port=3306,
                           charset='utf8')

# 使用cursor()方法获取操作游标
cur = db.cursor()
a = cur.execute("desc User_inf")
#1.查询操作
# 编写sql 查询语句  user 对应我的表名
sql = "select * from User_inf"
try:
	cur.execute(sql) 	#执行sql语句
 
	results = cur.fetchall()	#获取查询的所有记录
	print("id","borrow","repay")
	#遍历结果
	for row in results :
		id = row[0]
		name = row[-2].decode("utf-8")
		password = row[-1].decode("utf-8") 
		print(id,name,password)
except Exception as e:
	raise e
finally:
	db.close()	#关闭连接


id borrow repay
1435223 1  0 
172344 1  1 
18661377320 1  0 
1927233 1  1 
234567 1  0 
25233 1  1 
2667623 0  1 
34332423 0  1 
657851236 1  1 
7223654 0  0 
783423 0  1 


In [72]:
cur.description[0][0]

'User_id'

In [77]:
data.index.name = [0, 1, 2, 3,4, 5, 6]

In [78]:
data

,0,1,2,3,4,5,6
"[0, 1, 2, 3, 4, 5, 6]",,,,,,,
0,1435223,24.0,2.0,28.00,17.30,b'1\x00',0 
1,172344,40.0,15.0,120.00,2.90,b'1\x00',1 
2,18661377320,NaN,NaN,0.00,0.00,b'1\x00',0 
3,1927233,43.0,12.0,38.00,3.60,b'1\x00',1 
4,234567,41.0,17.0,176.00,9.30,b'1\x00',0 
5,25233,39.0,20.0,67.00,30.60,b'1\x00',1 
6,2667623,28.0,10.0,31.00,17.30,b'0\x00',1 
7,34332423,27.0,0.0,16.00,1.70,b'0\x00',1 
8,657851236,21.0,0.0,0.00,0.00,b'1\x00',1 


In [51]:
for i in range(len(data[6])):
    data.loc[6, i] = data[6, i].decode("utf-8") 

data[6]

KeyError: (6, 0)

In [8]:
from pandas import DataFrame
data  = DataFrame(list(results))
print (data.describe().T)

   count  mean       std   min    25%   50%    75%   max
1   10.0  34.0  8.151346  21.0  27.25  37.5  40.75  43.0
2   10.0   9.4  7.244922   0.0   2.75  11.0  14.50  20.0
